In [138]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import numpy as np
replacement_value = 'dummy_repalcement_value'
path = '/home/td/Documents'

# Problem structure

One way to set up the problem is to make it a binary classification task. This has the advantages of simplicity but it makes the assumption that large Makes share characteristics which is not always true. In this pass I solved the problem this way. 

A more thorough way to set up the problem would be to predict the make or a group of makes sharing similar characteristics and back into the probability. 

# Data problems

In [139]:
# Label problems
path = '/home/td/Documents'
df = pd.read_csv('{path}/tickets.csv'.format(path=path), low_memory=False)

top_25_makes = df['Make'].value_counts()[:25].index.tolist()
df['Make'].value_counts()[:25]

TOYT    721411
HOND    491961
FORD    382695
NISS    311324
CHEV    297076
BMW     199221
MERZ    177307
VOLK    149501
HYUN    133864
DODG    127764
LEXS    124508
KIA     101746
JEEP    100909
AUDI     84229
MAZD     79853
OTHR     72411
GMC      62391
CHRY     57317
INFI     56809
ACUR     52703
SUBA     46898
VOLV     42330
TOYO     40064
MITS     37842
CADI     34080
Name: Make, dtype: int64

Curation will be required, there is at least 1 duplicate with toyota and there are some invalid makes such as OTHR

In [140]:
df_valid_loc = df.copy()
df_valid_loc.loc[:,'Latidftude'] = df_valid_loc.loc[:,'Latitude'].replace(99999.0, np.nan)
df_valid_loc.loc[:,'Longitude'] = df_valid_loc.loc[:,'Longitude'].replace(99999.0, np.nan)

shape1 = df_valid_loc.shape[0]
df_valid_loc = df_valid_loc.dropna(subset = ['Latitude', 'Longitude'])
shape2 = df_valid_loc.shape[0]
perc_nan = (shape1 - shape2)/shape1
perc_nan
# Latitude and longitude had ~20% nan values, these values also do not map to DSM or decimal lat long formats.

0.1502618492246288

Record with ticket id 1109139006 should have latitude and longitude of 34.156940, -118.435250. It has 6467477, 1880027 instead. Provided lat and long don't seem to map to standard formats.


In [141]:
#TODO: check for differences in test and train set

# Label analysis

In [142]:
df_features = pd.read_csv('feature_analysis.csv')

In [143]:
df_features.sort_values('model_feature_importance', ascending = False)


,columns,intercept,model_feature_importance,p_value,r_value,slope,std_err
75,Issue time,0.497846,0.872696,0.344804,1.691957e-03,1.789388e-06,1.894100e-06
72,Fine amount,0.498704,0.015111,0.509448,1.181499e-03,1.849256e-05,2.803185e-05
74,plate_expiration_diff_ts,0.499560,0.010586,0.257164,-2.029384e-03,-3.013163e-22,2.659169e-22
73,lat_long_outlier_score,0.498300,0.009098,0.202947,2.280261e-03,2.046774e-03,1.607580e-03
30,route_dummy_replacement_value,0.499035,0.004858,0.586839,9.732532e-04,1.210285e-03,2.227154e-03
12,color_WT,0.500188,0.004827,0.681058,-7.361242e-04,-9.082704e-04,2.209797e-03
67,body_style_PA,0.498805,0.004347,0.620865,8.858589e-04,1.359198e-03,2.747934e-03
2,color_BK,0.499930,0.004332,0.882280,2.652064e-04,3.224668e-04,2.177657e-03
7,color_GY,0.500446,0.003950,0.310632,-1.815858e-03,-2.294626e-03,2.263172e-03
10,color_SL,0.500060,0.003673,0.858215,-3.199509e-04,-4.835582e-04,2.706785e-03


In [144]:
df_labels = pd.read_csv('label_analysis.csv')
df_labels = df_labels[df_labels['Make'].isin(top_25_makes)]
df_counts= df_labels['Make'].value_counts()
from sklearn.preprocessing import StandardScaler

relevant_features = ['Fine amount', 
                     'plate_expiration_diff_ts', 
                     'lat_long_outlier_score', 
                     'route_dummy_replacement_value',
                     'color_GY',
                     'color_BL',
                     'color_BK',
                     'color_WT',
                     'color_BL',
                     'color_SL',
                     'route_00600',
                     'route_00500']


for i in relevant_features:
    scaler = StandardScaler()
    df_labels.loc[:, i] =  scaler.fit_transform(df_labels[i].values.reshape(-1, 1))
df_labels


df_group = df_labels[['Make'] + relevant_features].groupby('Make').mean()

/home/td/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/td/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [145]:
from sklearn.cluster import KMeans


cluster_alg = KMeans(n_clusters = 2)
df_group['cluster'] = cluster_alg.fit_predict(df_group[relevant_features])
df_group.sort_values('cluster')


,Fine amount,plate_expiration_diff_ts,lat_long_outlier_score,route_dummy_replacement_value,color_GY,color_BL,color_BK,color_WT,color_BL,color_SL,route_00600,route_00500,cluster
Make,,,,,,,,,,,,,
ACUR,0.001659,-0.003236,-0.003544,0.001590,-0.001858,0.002586,-0.009768,0.009383,0.002586,-0.006129,-0.008421,-0.001067,0
TOYO,-0.020102,-0.002450,0.014309,-0.003473,-0.001646,0.002974,0.008510,0.000856,0.002974,-0.000633,-0.006795,-0.002440,0
NISS,-0.001246,-0.000193,-0.002837,-0.003980,-0.006371,0.003191,-0.000990,-0.000195,0.003191,0.009521,0.003793,0.000070,0
INFI,0.012707,-0.003742,0.007401,0.006225,-0.001384,0.007859,0.002304,0.003941,0.007859,-0.006404,-0.006553,0.008646,0
HYUN,-0.000987,-0.000440,0.000079,0.006151,-0.000142,0.005452,0.001276,-0.010277,0.005452,-0.002758,-0.000288,0.000713,0
HOND,0.001198,-0.000484,0.002897,-0.003177,-0.002195,0.002671,0.001417,0.000947,0.002671,-0.001682,0.001698,-0.000076,0
GMC,-0.002490,-0.001147,0.005344,-0.006107,0.003075,0.006808,-0.000285,-0.012287,0.006808,0.006798,0.001806,0.016590,0
JEEP,0.004187,-0.001816,0.005522,0.003216,0.000420,0.008180,-0.002034,-0.003658,0.008180,0.003661,-0.000489,0.002277,0
DODG,-0.006523,-0.002494,-0.000553,0.000095,-0.001184,0.007648,-0.003339,-0.002573,0.007648,0.000241,-0.011637,0.003638,0


In [146]:
df_group.groupby('cluster').mean()
        

,Fine amount,plate_expiration_diff_ts,lat_long_outlier_score,route_dummy_replacement_value,color_GY,color_BL,color_BK,color_WT,color_BL,color_SL,route_00600,route_00500
cluster,,,,,,,,,,,,
0,-0.002535,-0.001319,0.001906,0.001300,-0.000460,0.006628,-0.001602,-0.003095,0.006628,-0.000237,-0.002811,0.002261
1,-0.000691,-0.000957,-0.000300,-0.000452,0.002798,-0.004462,-0.000812,0.000889,-0.004462,-0.000311,0.000867,-0.000840


# Features

There are 3 general type of features: location, car characteristics and ticket/violation characteristics. 

The best location features are the Route and ticket location density (lat_long_outlier_score). 

The best car characteristics features are the Body type and the car color. 

The best violation features are the fine amount and time of the violation.

In [147]:
df_copy = df.copy()
df_copy = df_copy.dropna(subset = ['Make'])
df_copy['target'] = df_copy['Make'].isin(top_25_makes).astype(int)
df_copy.loc[:, 'ticket_dt'] = pd.to_datetime(df_copy['Issue Date'], errors='coerce')
df_copy.loc[:, 'plate_expiration_diff_dt'] = pd.to_datetime(df_copy['Plate Expiry Date'].astype(int, errors = 'ignore'), format = '%Y%m', errors='coerce')
df_copy.loc[:, 'plate_expiration_diff_dt'] = df_copy['plate_expiration_diff_dt'] - df['plate_expiration_diff_dt']
df_copy.loc[:, 'plate_expiration_diff_ts'] = df_copy['plate_expiration_diff_dt'].values.astype(np.int64)


KeyError: 'plate_expiration_diff_dt'

In [ ]:
top_states = df_copy['RP State Plate'].value_counts()[:8].index.tolist()
df_main_states = df_copy[df_copy['RP State Plate'].isin(top_states)]
df_main_states.groupby(['RP State Plate'])['target'].mean().sort_values()

In [ ]:
top_routes = df_copy['Route'].value_counts()[:8].index.tolist()
df_route = df_copy[df_copy['Route'].isin(top_routes)]
df_route.groupby(['Route'])['target'].mean().sort_values()

In [ ]:
top_body_styles = df_copy['Body Style'].value_counts()[:8].index.tolist()
df_body_styles = df_copy[df_copy['Body Style'].isin(top_body_styles)]
df_body_styles.groupby(['Body Style'])['target'].mean().sort_values()

In [ ]:
df.head()

In [ ]:
df_labels.corr().sort_values('lat_long_outlier_score')

# Evaluation